In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from SALib.sample import saltelli
from SALib.analyze import sobol

# Define the Ishigami function
def ishigami(x):
    return np.sin(x[0]) + 7 * np.sin(x[1])**2 + 0.1 * x[2]**4 * np.sin(x[0])

# Define the problem for Sobol sensitivity analysis
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-np.pi, np.pi]]*3
}

# Initialize samples and Sobol indices
param_values = np.empty((0, 3))
Y = np.empty((0,))
sobol_indices_over_time = {'ST': [[], [], []]}
mean_over_time = []
std_dev_over_time = []

plt.ion() # Turn on interactive mode

# Creating subplots with the right layout
fig, axs = plt.subplots(2, 3, figsize=(15, 10), gridspec_kw={'height_ratios': [3, 1], 'width_ratios': [1, 1, 1]})

# Looping for real-time plotting
for i in range(10, 1010, 10): # Incrementing by 10 samples
    # Adding 10 more samples using Monte Carlo sampling
    new_samples = np.random.uniform(-np.pi, np.pi, (10, 3))
    param_values = np.vstack([param_values, new_samples])
    new_Y = np.array([ishigami(v) for v in new_samples])
    Y = np.hstack([Y, new_Y])

    # Analyze Sobol indices (total order)
    Si = sobol.analyze(problem, Y, calc_second_order=False, print_to_console=False)

    # Calculate mean and 2-sigma standard deviation bounds
    mean_over_time.append(np.mean(Y))
    std_dev_over_time.append(2 * np.std(Y))

    # Storing Sobol indices for plotting
    for j in range(3):
        sobol_indices_over_time['ST'][j].append(Si['ST'][j])

    # Clearing the plots
    axs[0, 0].clear()
    axs[0, 1].clear()

    # Plotting Sobol indices (Total-order)
    for j, label in enumerate(['x1', 'x2', 'x3']):
        axs[0, 0].plot(sobol_indices_over_time['ST'][j], linestyle='--', label=f"Total Sobol Index for {label}")
    axs[0, 0].set_ylim(0, 1)
    axs[0, 0].set_title('Total Sobol Indices of the Ishigami Function')
    axs[0, 0].legend(loc="upper right")
    axs[0, 0].set_xlabel('Number of Iterations')

    # Plotting mean and 2-sigma standard deviation bounds
    axs[0, 1].plot(mean_over_time, label="Mean")
    axs[0, 1].fill_between(range(len(mean_over_time)), np.array(mean_over_time) - np.array(std_dev_over_time), np.array(mean_over_time) + np.array(std_dev_over_time), alpha=0.3, label="2-sigma bounds")
    axs[0, 1].set_title('Mean and 2-sigma bounds of the Ishigami Function')
    axs[0, 1].legend(loc="upper right")
    axs[0, 1].set_xlabel('Number of Iterations')

    # Clearing the axes for the bottom bar charts
    axs[1, 0].clear()
    axs[1, 1].clear()
    axs[1, 2].clear()

    # Plotting x1, x2, x3 values of the last sample
    axs[1, 0].bar(0, new_samples[-1, 0])
    axs[1, 0].set_ylim(-np.pi, np.pi)
    axs[1, 0].set_title('x1')

    axs[1, 1].bar(0, new_samples[-1, 1])
    axs[1, 1].set_ylim(-np.pi, np.pi)
    axs[1, 1].set_title('x2')

    axs[1, 2].bar(0, new_samples[-1, 2])
    axs[1, 2].set_ylim(-np.pi, np.pi)
    axs[1, 2].set_title('x3')    
    clear_output(wait=True)
    display(fig)

    plt.pause(0.1) # Pause for animation effect

plt.ioff() # Turn off interactive mode